In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import time as time
import re
from random import randint
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import logging

In [2]:
import sys
sys.path.append("../code")
import loaders.cityprotect as cp

Concatenate and pandaize data downloaded from CityProtect.
Given a folder, find all the files ending with "Los_Altos_Police_Department_report.csv", concatenate, and sort by date.

In [3]:
def geocode(geolocator, addr, sleep_sec):
    try:
        return geolocator.geocode(addr)
    except GeocoderTimedOut:
        logging.info('TIMED OUT: GeocoderTimedOut: Retrying...')
        time.sleep(randint(1*100,sleep_sec*100)/100)
        return geolocator.reverse_geocode(geolocator, latlon, sleep_sec)
    except GeocoderServiceError as e:
        logging.info('CONNECTION REFUSED: GeocoderServiceError encountered.')
        logging.error(e)
        return None
    except Exception as e:
        logging.info('ERROR: Terminating due to exception {}'.format(e))
        return None

In [4]:
# import module
from geopy.geocoders import Nominatim
  
# initialize Nominatim API 
user_agent = 'user_me_{}'.format(randint(10000,99999))
geolocator = Nominatim(user_agent=user_agent)
 

In [5]:
geolocator.geocode("1728 Selig Ln, Los Altos, CA")

GeocoderServiceError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:997)

In [5]:
previous_df = pd.read_pickle("../data/clean_data/SCCSheriff_2017_2022_inferredCity.pkl")

In [6]:
dept = "Santa Clara County Office of the Sheriff"
df = cp.cityprotect("../data/raw_data/{}".format(dept.replace(" ", "_")),
                                 start_date = previous_df["date"].max(),
                                  end_date = datetime.now() )

Loading from 26 files.
Data file ../data/raw_data/Santa_Clara_County_Office_of_the_Sheriff\Apr_Jun_2017_Santa_Clara_County_Office_of_the_Sheriff_report.csv is empty.
Data file ../data/raw_data/Santa_Clara_County_Office_of_the_Sheriff\Jan_Mar_2017_Santa_Clara_County_Office_of_the_Sheriff_report.csv is empty.


In [7]:
df["date"]

324514   2022-12-31 23:58:48
324515   2023-01-01 00:15:09
324516   2023-01-01 00:36:17
324517   2023-01-01 00:43:22
324518   2023-01-01 00:45:05
                 ...        
73254    2023-07-01 06:02:52
73255    2023-07-01 06:07:10
73256    2023-07-01 06:17:40
73244    2023-07-01 06:42:53
73257    2023-07-01 06:48:01
Name: date, Length: 22118, dtype: datetime64[ns]

In [8]:
df["inferredCity"]="unknown"

In [9]:
counter = 0
for index, row in df[df["inferredCity"] == "unknown"].iterrows():
    if row["inferredCity"] == "unknown":
        print(row)
        try:
            place = re.sub("BLOCK", "", row["blocksizedAddress"].upper()) + ", SANTA CLARA COUNTY, CA"
        except:
            print("Failed to scan: {}".format(row["blocksizedAddress"]))
            continue
        location = geocode(geolocator, place, 1)   
            
        counter = counter + 1
        if location != None:
            display_name = [s.strip() for s in location.raw["display_name"].split(",")]                         
            try:
                county_idx = display_name.index("Santa Clara County")
            except ValueError:
                print("SCC not found: {}".format(display_name))
                df.at[index, "inferredCity"] = "not_found"
                continue
            df.at[index, "inferredCity"] = display_name[county_idx-1]   
            df.at[index, "postalCode"] = display_name[-2]                            
            print("\n{} {}: {}, {}".format(counter, location.raw["display_name"], df.at[index,"inferredCity"], df.at[index, "postalCode"]))
        else:
            df.at[index, "inferredCity"] = "no resolution"
            print("NR: {} ".format(counter), end="")
        if counter/1000 == round(counter/1000):
            print("Saving...")
            df.to_pickle("SCCSheriff.pkl")
    else:
        print(".", end="")


ERROR:root:[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:997)


ccn                                                          S223650282
date                                                2022-12-31 23:58:48
updateDate                                          2023-01-01 09:05:07
city                                                 SANTA CLARA COUNTY
state                                                                CA
postalCode                                                            .
blocksizedAddress                               14000 Block LOMA RIO DR
incidentType                                       PHONE UR OFFICE, OR:
parentIncidentType                                                Other
narrative             Call Type: 1021   <br>Description: PHONE UR OF...
inferredCity                                                    unknown
Name: 324514, dtype: object
NR: 1 ccn                                                          L223650085
date                                                2023-01-01 00:15:09
updateDate                    

ERROR:root:[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:997)
ERROR:root:[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:997)


NR: 2 ccn                                                          L223650088
date                                                2023-01-01 00:36:17
updateDate                                          2023-01-01 09:05:08
city                                                 SANTA CLARA COUNTY
state                                                                CA
postalCode                                                            .
blocksizedAddress                                             BONINO LN
incidentType                                     SERVICE OR AID REQUEST
parentIncidentType                                                Other
narrative             Call Type: SVC    <br>Description: SERVICE OR ...
inferredCity                                                    unknown
Name: 324516, dtype: object
NR: 3 ccn                                                          L223650094
date                                                2023-01-01 00:43:22
updateDate              

ERROR:root:[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:997)
ERROR:root:[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:997)


...

ERROR:root:[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:997)
ERROR:root:[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:997)


NR: 18 ccn                                                          S230010024
date                                                2023-01-01 08:07:36
updateDate                                          2023-01-01 17:51:41
city                                                 SANTA CLARA COUNTY
state                                                                CA
postalCode                                                            .
blocksizedAddress                                15000 Block FREEMAN AV
incidentType                                                DISTURBANCE
parentIncidentType                                      Quality of Life
narrative             Call Type: 415    <br>Description: DISTURBANCE...
inferredCity                                                    unknown
Name: 174914, dtype: object
NR: 19 ccn                                                          S230010025
date                                                2023-01-01 08:08:40
updateDate            

ERROR:root:[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:997)


NR: 20 ccn                                                          S230010027
date                                                2023-01-01 08:11:47
updateDate                                          2023-01-01 17:51:40
city                                                 SANTA CLARA COUNTY
state                                                                CA
postalCode                                                            .
blocksizedAddress                               10600 Block W LOYOLA DR
incidentType                                             ALARM, AUDIBLE
parentIncidentType                                                Other
narrative             Call Type: 1033A  <br>Description: ALARM, AUDI...
inferredCity                                                    unknown
Name: 174916, dtype: object


NameError: name 'sleep' is not defined

In [ ]:
df.to_pickle("SCCSheriff.pkl")

In [ ]:
df["inferredCity"].value_counts()

In [ ]:
df[df["inferredCity" == "unknown"]]

        try: 
            location = geolocator.geocode(place)
        except GeocoderTimedOut as e:
            print("Location error: {}".format(e))
            naptime = 3+randint(100)
            print("Sleeping {} seconds.".format(naptime))
            time.sleep(naptime)
            user_agent = 'user_me_{}'.format(randint(10000,99999))
            geolocator = Nominatim(user_agent=user_agent)
            location = geolocator.geocode(place)
